# Pressure Data

We can look at the raw pressure data from the BMP180 seonsor, with only a naïve correction to local pressure-altitude of the launch site:

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import launch

In [ ]:
alt_time, imualt = launch.cached_altitude()

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated")
plt.ylabel(r"Pressure [kPa]")
plt.xlabel(r"Mission Elapsed Time [s]")

ax1.plot(launch.bmp1.time, launch.bmp1.pressure, alpha=0.75, label="Pressure Sensor")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Altitude [km]")
ax2.plot(alt_time, divide(imualt, 1000.0), 'k-', alpha=0.18, label="IMU Integrated Altitude")
ax2.grid(b=False)

plt.xlim([-5,42])
ax2.legend(loc=1)
ax1.legend(loc=3)
plt.show()

The Pressure goes down and the rocket goes up, as expected.

## Temperature

The BMP180 also has a temperature sensor onboard to do temperature corrections for pressure value. The temperature of the device appears to have been relativly constant during the vertical flight.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated Temperature")
plt.ylabel(r"Temperature [${}^0$C]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.bmp1.time, launch.bmp1.temperature, alpha=0.75)

plt.xlim([-5,42])
plt.show()

## Altitude

Even with a rough guess of the pressure, we can apply an exponetial atmosphere and see what altitude the pressure sensor suggests we went to:

In [ ]:
sealevel_pa=101.325
altitudes = []
for p in launch.bmp1.pressure:
    altitude = 44330.0 * (1.0 - pow(p / sealevel_pa, (1.0/5.255)))
    altitude = altitude - 1390
    altitudes.append(altitude)

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated Altitude (Exponental Atmosphere, No Temperature Correction)")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.bmp1.time, divide(altitudes, 1000.0), alpha=0.75, label="Pressure Sensor")
plt.plot(alt_time, divide(imualt, 1000.0), alpha=0.5, label="IMU Integrated Altitude")

plt.xlim([-5,42])
ax1.legend(loc=1)
plt.show()

The large diversion suggests a need for calibration.